In [ ]:
%load_ext cudf

The cudf module is not an IPython extension.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

## IVT 31

In [ ]:
df_31_IVT = pd.read_csv('data/STData/31/31_IVT.csv')

In [ ]:
df_31_IVT.head()

In [ ]:
df_31_IVT.columns

In [ ]:
df_31_IVT.shape

In [ ]:
df_31_IVT.info()

In [ ]:
df_31_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_31_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_31_IVT['QuestionKey'].unique()

In [ ]:
df_31_IVT['Timestamp'] = pd.to_datetime(df_31_IVT['Timestamp'])

In [ ]:
df_31_IVT.head(3)

In [ ]:
df_31_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_31_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_31_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_31_IVT.isnull().sum()

In [ ]:
df_31_IVT.head()

In [ ]:
df_31_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_31_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_31_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_31_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_31_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_31_IVT.describe()

In [ ]:
df_31_IVT.head(3)

In [ ]:
df_31_IVT['Timestamp'] = pd.to_datetime(df_31_IVT['Timestamp'])

In [ ]:
df_31_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_31_IVT['Timestamp'], y=df_31_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_31_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_31_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_31_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_31_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_31_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_31_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_31_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_31_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_31_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_31_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_31_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_31_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_31_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_31_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_31_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_31_IVT.columns

In [ ]:
fix_1_df = df_31_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_31_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 32

In [ ]:
df_32_IVT = pd.read_csv('data/STData/32/32_IVT.csv')

In [ ]:
df_32_IVT.head()

In [ ]:
df_32_IVT.columns

In [ ]:
df_32_IVT.shape

In [ ]:
df_32_IVT.info()

In [ ]:
df_32_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_32_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_32_IVT['QuestionKey'].unique()

In [ ]:
df_32_IVT['Timestamp'] = pd.to_datetime(df_32_IVT['Timestamp'])

In [ ]:
df_32_IVT.head(3)

In [ ]:
df_32_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_32_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_32_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_32_IVT.isnull().sum()

In [ ]:
df_32_IVT.head()

In [ ]:
df_32_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_32_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_32_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_32_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_32_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_32_IVT.describe()

In [ ]:
df_32_IVT.head(3)

In [ ]:
df_32_IVT['Timestamp'] = pd.to_datetime(df_32_IVT['Timestamp'])

In [ ]:
df_32_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_32_IVT['Timestamp'], y=df_32_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_32_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_32_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_32_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_32_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_32_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_32_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_32_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_32_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_32_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_32_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_32_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_32_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_32_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_32_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_32_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_32_IVT.columns

In [ ]:
fix_1_df = df_32_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_32_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 33

In [ ]:
df_33_IVT = pd.read_csv('data/STData/33/33_IVT.csv')

In [ ]:
df_33_IVT.head()

In [ ]:
df_33_IVT.columns

In [ ]:
df_33_IVT.shape

In [ ]:
df_33_IVT.info()

In [ ]:
df_33_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_33_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_33_IVT['QuestionKey'].unique()

In [ ]:
df_33_IVT['Timestamp'] = pd.to_datetime(df_33_IVT['Timestamp'])

In [ ]:
df_33_IVT.head(3)

In [ ]:
df_33_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_33_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_33_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_33_IVT.isnull().sum()

In [ ]:
df_33_IVT.head()

In [ ]:
df_33_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_33_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_33_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_33_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_33_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_33_IVT.describe()

In [ ]:
df_33_IVT.head(3)

In [ ]:
df_33_IVT['Timestamp'] = pd.to_datetime(df_33_IVT['Timestamp'])

In [ ]:
df_33_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_33_IVT['Timestamp'], y=df_33_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_33_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_33_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_33_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_33_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_33_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_33_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_33_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_33_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_33_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_33_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_33_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_33_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_33_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_33_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_33_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_33_IVT.columns

In [ ]:
fix_1_df = df_33_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_33_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 34

In [ ]:
df_34_IVT = pd.read_csv('data/STData/34/34_IVT.csv')

In [ ]:
df_34_IVT.head()

In [ ]:
df_34_IVT.columns

In [ ]:
df_34_IVT.shape

In [ ]:
df_34_IVT.info()

In [ ]:
df_34_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_34_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_34_IVT['QuestionKey'].unique()

In [ ]:
df_34_IVT['Timestamp'] = pd.to_datetime(df_34_IVT['Timestamp'])

In [ ]:
df_34_IVT.head(3)

In [ ]:
df_34_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_34_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_34_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_34_IVT.isnull().sum()

In [ ]:
df_34_IVT.head()

In [ ]:
df_34_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_34_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_34_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_34_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_34_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_34_IVT.describe()

In [ ]:
df_34_IVT.head(3)

In [ ]:
df_34_IVT['Timestamp'] = pd.to_datetime(df_34_IVT['Timestamp'])

In [ ]:
df_34_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_34_IVT['Timestamp'], y=df_34_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_34_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_34_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_34_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_34_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_34_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_34_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_34_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_34_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_34_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_34_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_34_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_34_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_34_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_34_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_34_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_34_IVT.columns

In [ ]:
fix_1_df = df_34_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_34_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 35

In [ ]:
df_35_IVT = pd.read_csv('data/STData/35/35_IVT.csv')

In [ ]:
df_35_IVT.head()

In [ ]:
df_35_IVT.columns

In [ ]:
df_35_IVT.shape

In [ ]:
df_35_IVT.info()

In [ ]:
df_35_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_35_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_35_IVT['QuestionKey'].unique()

In [ ]:
df_35_IVT['Timestamp'] = pd.to_datetime(df_35_IVT['Timestamp'])

In [ ]:
df_35_IVT.head(3)

In [ ]:
df_35_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_35_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_35_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_35_IVT.isnull().sum()

In [ ]:
df_35_IVT.head()

In [ ]:
df_35_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_35_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_35_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_35_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_35_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_35_IVT.describe()

In [ ]:
df_35_IVT.head(3)

In [ ]:
df_35_IVT['Timestamp'] = pd.to_datetime(df_35_IVT['Timestamp'])

In [ ]:
df_35_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_35_IVT['Timestamp'], y=df_35_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_35_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_35_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_35_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_35_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_35_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_35_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_35_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_35_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_35_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_35_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_35_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_35_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_35_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_35_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_35_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_35_IVT.columns

In [ ]:
fix_1_df = df_35_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_35_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 36

In [ ]:
df_36_IVT = pd.read_csv('data/STData/36/36_IVT.csv')

In [ ]:
df_36_IVT.head()

In [ ]:
df_36_IVT.columns

In [ ]:
df_36_IVT.shape

In [ ]:
df_36_IVT.info()

In [ ]:
df_36_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_36_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_36_IVT['QuestionKey'].unique()

In [ ]:
df_36_IVT['Timestamp'] = pd.to_datetime(df_36_IVT['Timestamp'])

In [ ]:
df_36_IVT.head(3)

In [ ]:
df_36_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_36_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_36_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_36_IVT.isnull().sum()

In [ ]:
df_36_IVT.head()

In [ ]:
df_36_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_36_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_36_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_36_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_36_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_36_IVT.describe()

In [ ]:
df_36_IVT.head(3)

In [ ]:
df_36_IVT['Timestamp'] = pd.to_datetime(df_36_IVT['Timestamp'])

In [ ]:
df_36_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_36_IVT['Timestamp'], y=df_36_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_36_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_36_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_36_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_36_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_36_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_36_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_36_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_36_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_36_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_36_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_36_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_36_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_36_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_36_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_36_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_36_IVT.columns

In [ ]:
fix_1_df = df_36_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_36_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 37

In [ ]:
df_37_IVT = pd.read_csv('data/STData/37/37_IVT.csv')

In [ ]:
df_37_IVT.head()

In [ ]:
df_37_IVT.columns

In [ ]:
df_37_IVT.shape

In [ ]:
df_37_IVT.info()

In [ ]:
df_37_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_37_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_37_IVT['QuestionKey'].unique()

In [ ]:
df_37_IVT['Timestamp'] = pd.to_datetime(df_37_IVT['Timestamp'])

In [ ]:
df_37_IVT.head(3)

In [ ]:
df_37_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_37_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_37_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_37_IVT.isnull().sum()

In [ ]:
df_37_IVT.head()

In [ ]:
df_37_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_37_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_37_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_37_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_37_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_37_IVT.describe()

In [ ]:
df_37_IVT.head(3)

In [ ]:
df_37_IVT['Timestamp'] = pd.to_datetime(df_37_IVT['Timestamp'])

In [ ]:
df_37_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_37_IVT['Timestamp'], y=df_37_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_37_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_37_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_37_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_37_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_37_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_37_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_37_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_37_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_37_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_37_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_37_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_37_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_37_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_37_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_37_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_37_IVT.columns

In [ ]:
fix_1_df = df_37_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_37_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

## IVT 38

In [ ]:
df_38_IVT = pd.read_csv('data/STData/38/38_IVT.csv')

In [ ]:
df_38_IVT.head()

In [ ]:
df_38_IVT.columns

In [ ]:
df_38_IVT.shape

In [ ]:
df_38_IVT.info()

In [ ]:
df_38_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_38_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_38_IVT['QuestionKey'].unique()

In [ ]:
df_38_IVT['Timestamp'] = pd.to_datetime(df_38_IVT['Timestamp'])

In [ ]:
df_38_IVT.head(3)

In [ ]:
df_38_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_38_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_38_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_38_IVT.isnull().sum()

In [ ]:
df_38_IVT.head()

In [ ]:
df_38_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_38_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_38_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_38_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_38_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_38_IVT.describe()

In [ ]:
df_38_IVT.head(3)

In [ ]:
df_38_IVT['Timestamp'] = pd.to_datetime(df_38_IVT['Timestamp'])

In [ ]:
df_38_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_38_IVT['Timestamp'], y=df_38_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_38_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_38_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_38_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_38_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_38_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_38_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_38_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_38_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_38_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_38_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_38_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_38_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_38_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_38_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_38_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_38_IVT.columns

In [ ]:
fix_1_df = df_38_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_38_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.